# A Quadrature Signals Tutorial, Complex, But Not Complicated
https://mriquestions.com/uploads/3/4/5/7/34572113/quad_signals_tutorial-lyons.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', 'Waves', 'bitbucket', 'github', 'codes'],
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from waves.sound import *
from waves.tools.dsp.mixers.frequency_mixers import *

Populating the interactive namespace from numpy and matplotlib


In [5]:
import warnings 

warnings.filterwarnings('ignore')

In [6]:
def show_pair(sig_left, sig_right, frames = 300):
    sig_left.show(color = 'red', show_frames = slice(0, frames))
    sig_right.show(color = 'green', show_frames = slice(0, frames))

## LO

In [7]:
phase_shift = PI2 / 8

In [8]:
freq_lo = int(100e3)

freq_sampling = freq_lo * 8 * 2

In [9]:
duration_seconds = 1
t = np.linspace(0, duration_seconds, duration_seconds * freq_sampling)

$
\begin{align}
\cos(2\pi f_o t ) &=  \frac{e^{-j2\pi f_ot} + e^{j2\pi f_ot} }{2} \\
                  &=  \frac{e^{-j2\pi f_ot}}{2} + \frac{e^{j2\pi f_ot} }{2} \\ 
\sin(2\pi f_o t ) &=  \frac{e^{j 2\pi f_ot} - e^{-j2\pi f_ot} }{2j} \\
                  &=  j\frac{e^{-j2\pi f_ot}}{2} -j \frac{e^{j2\pi f_ot} }{2} \\
signal            &= e^{j 2\pi \omega t} \\
                  &= \cos(2\pi \omega t ) + j\sin(2\pi \omega t ) \\
                  &= \frac{e^{-j2\pi \omega t}}{2} + \frac{e^{j2\pi \omega t} }{2} +j(j\frac{e^{-j2\pi \omega t}}{2} -j \frac{e^{j2\pi \omega t} }{2}) \\
                  &= \frac{e^{-j2\pi \omega t}}{2} + \frac{e^{j2\pi \omega t} }{2} -\frac{e^{-j2\pi \omega t}}{2} + \frac{e^{j2\pi \omega t} }{2} \\
                  &=    e^{j2\pi \omega t} \\
signal \cdot \cos(2\pi f_o t ) &= e^{j 2\pi \omega t} \cdot (\frac{e^{-j2\pi f_ot}}{2} + \frac{e^{j2\pi f_ot} }{2}) \\ 
                  &=  \frac{e^{-j2\pi t(f_o - \omega )}}{2} + \frac{e^{j2\pi t ( f_o + \omega )} }{2} \\
signal \cdot \sin(2\pi f_o t ) &= e^{j 2\pi \omega t} \cdot (j\frac{e^{-j2\pi f_ot}}{2} -j \frac{e^{j2\pi f_ot} }{2}) \\ 
                  &=  j\frac{e^{-j2\pi t( f_o - \omega)}}{2} -j \frac{e^{j2\pi t (f_o + \omega)} }{2} \\
IF                &= signal \cdot \cos(2\pi f_o t ) + signal \cdot \sin(2\pi f_o t ) \\
                  &= \frac{e^{-j2\pi t(f_o - \omega )}}{2} + \frac{e^{j2\pi t ( f_o + \omega )} }{2} +  j\frac{e^{-j2\pi t( f_o - \omega)}}{2} -j \frac{e^{j2\pi t (f_o + \omega)} }{2} \\
                  &= \frac{e^{-j2\pi t(f_o - \omega )}}{2} +  j\frac{e^{-j2\pi t( f_o - \omega)}}{2} + \frac{e^{j2\pi t ( f_o + \omega )} }{2}-j \frac{e^{j2\pi t (f_o + \omega)} }{2} \\
                  &= e^{-j2\pi t(f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t ( f_o + \omega )} (\frac{1 - j}{2}) \\
I                 &= IF \cdot \cos(2\pi f_o t) \\
                  &= (e^{-j2\pi t(f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t ( f_o + \omega )} (\frac{1 - j}{2})) \cdot \cos(2\pi f_o t) \\ 
                  &= (e^{-j2\pi t(f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t ( f_o + \omega )} (\frac{1 - j}{2})) \cdot \frac{{(  e^{-j2\pi f_ot}  + e^{j2\pi f_ot} })}{2} \\                   
                  &= \frac{(e^{-j2\pi t(f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t ( f_o + \omega )} (\frac{1 - j}{2})) \cdot ( e^{-j2\pi f_ot}  +  e^{j2\pi f_ot} )} {2}\\ 
                  &= \frac{(e^{-j2\pi t(2f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t ( \omega )} (\frac{1 - j}{2}) + e^{-j2\pi t(  - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t( 2f_o + \omega )}(\frac{1 - j}{2}))} {2}\\                  
                  &= \frac{(e^{-j2\pi t(2f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t  \omega} (\frac{1 - j}{2}) + e^{j2\pi t\omega }(\frac{1 + j}{2}) + e^{j2\pi t( 2f_o + \omega )}(\frac{1 - j}{2}))} {2}\\ 
                  &= \frac{(e^{-j2\pi t(2f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t  \omega}   + e^{j2\pi t( 2f_o + \omega )}(\frac{1 - j}{2}))} {2}\\ 
I_{low_pass}      &\approx \frac{e^{j2\pi    \omega t } }{4}  \\
Q                 &= IF \cdot \sin(2\pi f_o t) \\
                  &= (e^{-j2\pi t(f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t ( f_o + \omega )} (\frac{1 - j}{2})) \cdot ( j\frac{e^{-j2\pi f_ot}}{2} -j \frac{e^{j2\pi f_ot} }{2}) \\ 
                  &= \frac{(e^{-j2\pi t(f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t ( f_o + \omega )} (\frac{1 - j}{2})) \cdot (  e^{-j2\pi f_ot} - e^{j2\pi f_ot} )}{2} j\\    
                  &= \frac{(e^{-j2\pi t(2f_o - \omega )}(\frac{1 + j}{2}) + e^{j2\pi t  \omega} -e^{-j2\pi t(2f_o - \omega )}(\frac{1 + j}{2}) }{2} j\\       
Q_{low_pass}      &\approx \frac{e^{j2\pi    \omega t } }{4} j \\
(I_{low_pass} -j Q_{low_pass}) \cdot 2 &=  (\frac{e^{j2\pi    \omega t } }{4} + \frac{e^{j2\pi    \omega t } }{4} ) \cdot 2 =  e^{j2\pi    \omega t } = original \space signal
\end{align}
$  